# The return of the generator

Presentation to the San Diego Python User Group, June 2022, by Erik Colban  
(Available at: https://github.com/ecolban/SDPUG/tree/master/the_return_of_the_generator)

A generator function is a function that returns a generator. A generator yields values. A much less know fact about generators is that they also _return_ a value.

Let's start with some regular generators that apparently don't return anything.

**Example:**

In [1]:
def nats(start=0):
    n = start
    while True:
        yield n
        n += 1

In [2]:
g = nats(1)
[next(g) for _ in range(10)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
[next(g) for _ in range(10)]

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

To "reset" a generator, we can re-assign `g` to a new instance (and let the old one be garbage-collected).

In [4]:
g = nats(1)
[next(g) for _ in range(10)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

A generator like this one never returns. To see what happens when a generator returns, we can define a generator that yields a finite number of values.

**Example:**

In [5]:
def nats_bounded(start, end):
    for n in range(start, end):
        yield n

In [6]:
list(nats_bounded(1, 11))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

If we call `next` on the generator more times than the number of values that the generator yields, a `StopIteration` exception is raised. So it doesn't "return" in the same sense as when a regular function returns.

In [7]:
g = nats_bounded(1, 11)
[next(g) for _ in range(11)]

StopIteration: 

What happens if we add a return statement?

In [10]:
def nats_bounded_accummulate(start, end):
    total = 0
    n = start
    for n in range(start, end):
        total += n
        yield n        
    return total

In [9]:
g = nats_bounded_accummulate(1, 11)
print([next(g) for _ in range(11)])

StopIteration: 55

Notice that the return value is passed as the `StopIteration`'s value. This shows that we can have generators that both yield values and return a value. If we want the returned value, "all" we need to do is to catch the `StopIteration` exception and retrieve the returned value from it.

In [11]:
g = nats_bounded_accummulate(1, 11)
try:
    print([next(g) for _ in range(11)])
except StopIteration as e:
    print(f'The generator returned {e.value}.')

The generator returned 55.


However, there is a simpler way to get to the returned value. The expression `yield from g`, where `g` is a generator, evaluates to the value returned by `g`.

In [12]:
def from_generator(g):
    value = yield from g
    yield value
    
list(from_generator(nats_bounded_accummulate(1, 11)))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 55]

But is there any practical use case for this? Here is an example of how it can be used. See [on_itertools_groupby](https://github.com/ecolban/SDPUG/tree/master/on_itertools_groupby) for a more elaborate example.

**Example:**

In [13]:
from itertools import groupby

def sum_all(nums):
    grand_total = 0
    for k, g in groupby(nums, key=lambda n: n // 10):
        grand_total += yield from sum_group(k, g)
    yield f'Total = {grand_total}'
    

def sum_group(key, g):
    a = list(g)
    group_total = sum(a)
    yield f"Group {key}: {' + '.join(str(n) for n in a)} = {group_total}"
    return group_total

for s in sum_all(range(18, 45)):
    print(s)

Group 1: 18 + 19 = 37
Group 2: 20 + 21 + 22 + 23 + 24 + 25 + 26 + 27 + 28 + 29 = 245
Group 3: 30 + 31 + 32 + 33 + 34 + 35 + 36 + 37 + 38 + 39 = 345
Group 4: 40 + 41 + 42 + 43 + 44 = 210
Total = 837


### Afterthoughts

But, why not just make these functions into ordinary functions?

In [ ]:
from itertools import groupby

def sum_all_with_side_effects(nums):
    grand_total = 0
    for k, g in groupby(nums, key=lambda n: n // 10):
        grand_total += sum_group_with_side_effects(k, g)
    print( f'Total = {grand_total}')
    
def sum_group_with_side_effects(key, g):
    a = list(g)
    group_total = sum(a)
    print( f"Group {key}: {' + '.join(str(n) for n in a)} = {group_total}")
    return group_total


sum_all_with_side_effects(range(18, 45))

First, by having generator functions, we postpone the decision on what we do with the values that are yielded; we might write them to a file, join them into a string, or, like here, print them to the console.

Second, a print statement is a side-effect. Functions that have no side-effects (called pure) are very easy to test; simply verify that for given input, they return the expected output.

Eventually, one wants to see some effect. A program with no effects does nothing. So, the point is not to avoid all side-effects, but to push out the responsibility of producing them as far as possible (to the caller's caller's ... caller). In this example, we have pushed the responsibility of producing the desired effect to the top, which is so trivial that it requires no testing (unless one doesn't trust that `print` does what it is expected to do).

## Testing generator functions

In [ ]:
def test_sum_group_yield():
    assert list(sum_group(1, [10, 13])) == ['Group 1: 10 + 13 = 23']


def test_sum_group_return():
    try:
        g = sum_group(1, [10, 13])
        while True:
            next(g)
    except StopIteration as e:
        assert e.value == 23


def test_sum_all_yield():
    assert list(sum_all([])) == ['Total = 0']
    assert list(sum_all([1, 2, 3])) == [
        'Group 0: 1 + 2 + 3 = 6',
        'Total = 6',
    ]
    assert list(sum_all([1, 3, 13, 16, 33])) == [
        'Group 0: 1 + 3 = 4',
        'Group 1: 13 + 16 = 29',
        'Group 3: 33 = 33',
        'Total = 66',
    ]


def test_sum_all_return():
    try:
        g = sum_all([])
        while True:
            next(g)
    except StopIteration as e:
        assert e.value is None


test_sum_group_yield()
test_sum_group_return()
test_sum_all_yield()
test_sum_all_return()

## Summary / Poll

Did you already know ...

1. about generator functions and generators? Yes = 6, No = 5
2. you can have a `return` statement in a generator function? Yes = 5, No = 6
3. generators return a value in addition to yielding values? Yes = 5, No = 6
4. the returned value of a generator is passed in the `StopIteration`'s `value` attribute? Yes = 4, No = 7
5. the expression `yield from g` evaluates to the value returned by `g`? Yes = 2, No = 9 
6. generator functions can be used to avoid side-effects (i.e., the responsibility of producing desired effects is passed to the caller)? Yes = 3, No = 8